# Calculator Applet

Demo that does various calculations - sha, hmac, pbkdf, field math, elliptic curve math

In [1]:
from applets.helpers import get_connection, AppletBase

class Calculator(AppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CC01", connection)

    def sha256(self, d):
        data = bytes([len(d)])+d
        r = "B0A10000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha256(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A20000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def sha512(self, d):
        data = bytes([len(d)])+d
        r = "B0A30000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha512(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A40000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

    def add_mod_FP(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A50000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def add_mod_N(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A60000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

In [2]:
connection = get_connection()
if connection is not None:
    atr=bytes(connection.getATR())
    print(atr.hex())
else:
    print("Failed to open a connection. No cardreaders?")

3bdc18ff8191fe1fc38073c821136605036351000250


In [3]:
app = Calculator(connection)
app.select()

b''

In [4]:
d = b'hello'
k = b'key'

In [5]:
# sha functions from the card
print("sha256:\t\t", app.sha256(d).hex())
print("hmac-sha256:\t", app.hmac_sha256(k, d).hex())
print("sha512:\t\t", app.sha512(d).hex())
print("hmac-sha512:\t", app.hmac_sha512(k, d).hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [6]:
import hashlib, hmac
# same using hashlib
print("sha256:\t\t",hashlib.sha256(d).digest().hex())
print("hmac-sha256:\t",hmac.new(k, d, digestmod=hashlib.sha256).digest().hex())
print("sha512:\t\t",hashlib.sha512(d).digest().hex())
print("hmac-sha512:\t",hmac.new(k, d, digestmod=hashlib.sha512).digest().hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [7]:
FP = 2**256 - 2**32 - 977
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

In [8]:
def to_bytes(n:int):
    return n.to_bytes(32, 'big')

In [9]:
# random check
import random
for i in range(10):
    a = random.randint(1,N)
    b = random.randint(1,N)
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

0
1
2
3
4
5
6
7
8
9


In [10]:
# edge cases - slightly less than mod, slightly more, equal and overflow
a = FP-20
for i, b in enumerate([10,23,20,10500, 1020203124]):
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

0
1
2
3
4


In [11]:
from embit import bip32, ec
import os
seed = os.urandom(64)
root = bip32.HDKey.from_seed(seed)
# without prefix, depth, root fingerprint and index
root.serialize()[13:].hex()

'c37f2c6a6fbf3fba8acc1a215c4a8c7be9344f9a2df66ac39fd3e0f61a76b2f0005fc6fc26f47947fcb4a0c8df669cf985970807e4fd42d33a788008af3a53346d'

In [12]:
res = root.child(10, hardened=False).serialize()[13:].hex()
res

'2b62bda2f6ac2082ad00b44eac546a71973c005c42c19404c6cce603bb60d51000b7b61abcb39e34f300f7ad23646a9ebc2c13de13385ac84ea1f2672d5ffce6ca'

In [13]:
data = bytes([65])+root.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0A90000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'2b62bda2f6ac2082ad00b44eac546a71973c005c42c19404c6cce603bb60d51000b7b61abcb39e34f300f7ad23646a9ebc2c13de13385ac84ea1f2672d5ffce6ca'

In [14]:
# point addition
p = root.to_public().key
p.compressed = False
print(p.sec().hex())
pp = ec.PublicKey.parse(p.serialize())
ec.secp256k1.ec_pubkey_tweak_add(pp._point, b'4'*32)
res = pp.sec().hex()
print(res)

043e135df75c7777225d1d0c85f1d0873e586870ce7366d2b9467a4c505a77e2e30de124097e55189b5f553d645097fa1341478afa4e4959863c57c25f3903233e
041346f49a6e0a7e0a35eaf4314176d194ecd63a05ac985167357ca77ecafefb0a6635dd422d979077631efe8744c77b4476ce954f9c9288c495832769d303a8fa


In [15]:
data = bytes([32])+b'4'*32+bytes([65])+p.sec()
r = "B0A70000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'041346f49a6e0a7e0a35eaf4314176d194ecd63a05ac985167357ca77ecafefb0a6635dd422d979077631efe8744c77b4476ce954f9c9288c495832769d303a8fa'

In [16]:
p3 = ec.PublicKey.parse(ec.secp256k1.ec_pubkey_serialize(ec.secp256k1.ec_pubkey_combine(p._point,pp._point)))
p3.compressed = False
res = p3.sec().hex()
res

'04be61dcce19e885df5146248a7fb41ce036a18aa4a31296819a6546d1a745a212b73decfa08efb7f317785669191c4198ac8ae198d8dd941efe3ad1ebe5ce4892'

In [17]:
data = bytes([65])+pp.sec()+bytes([65])+p.sec()
r = "B0A80000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

'04be61dcce19e885df5146248a7fb41ce036a18aa4a31296819a6546d1a745a212b73decfa08efb7f317785669191c4198ac8ae198d8dd941efe3ad1ebe5ce4892'

In [18]:
xpub = root.to_public()
xpub.serialize()[13:].hex()

'c37f2c6a6fbf3fba8acc1a215c4a8c7be9344f9a2df66ac39fd3e0f61a76b2f0023e135df75c7777225d1d0c85f1d0873e586870ce7366d2b9467a4c505a77e2e3'

In [19]:
res=xpub.child(10, hardened=False).serialize()[13:].hex()
res

'2b62bda2f6ac2082ad00b44eac546a71973c005c42c19404c6cce603bb60d510020faf3c13f7ce1c60fd6b04a4fe793b5ce0d1d8ae091e4520a6875ad24b82efa2'

In [20]:
data = bytes([65])+xpub.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0AA0000"+(bytes([len(data)])+data).hex().upper()
res2=app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh",res2)
res2

'2b62bda2f6ac2082ad00b44eac546a71973c005c42c19404c6cce603bb60d510020faf3c13f7ce1c60fd6b04a4fe793b5ce0d1d8ae091e4520a6875ad24b82efa2'

In [21]:
import hashlib
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 2048, 64).hex())
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 100, 64).hex())

82218bfe339fa8a349423df85eaf89728b4e7dc02360e8c6feec346a8e46232db544f6f2d2fff1430e53f76c0adafd9a2087652b7b943fdc07c0cff4c74252e2
038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f


In [22]:
import time
time.time()

1586798631.152561

In [23]:
t0 = time.time()
# 2048 iterations -> 0x8000
iterations = 100
data = bytes([2])+iterations.to_bytes(2,'big')+bytes([3])+b'pwd'+bytes([4])+b'salt'
r = "B0A00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())
dt = time.time()-t0
print(f"{dt} seconds for {iterations} iterations.\nFor 2048 iterations it will be {dt*2048/iterations} seconds")

038c39d10762fd77de3a035e2c8c7ba4410f07f8251a28b1d670649abc256e810afb12e6c7e4b3e976fc1be1a387c345b282c9127f02b23862c16859bde9ff7f
3.285496950149536 seconds for 100 iterations.
For 2048 iterations it will be 67.2869775390625 seconds


In [24]:
import os
a = os.urandom(32)
pow(int.from_bytes(a,'big'),2,FP).to_bytes(32,'big').hex()

'48d7086d5b10b8ef1b7f1e293b64e273ee4cdb551c1ec28e07d88a4996388ece'

In [25]:
data = bytes([32])+a
r = "B0AB0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


48d7086d5b10b8ef1b7f1e293b64e273ee4cdb551c1ec28e07d88a4996388ece


In [26]:
pow(int.from_bytes(a,'big'),3,FP).to_bytes(32,'big').hex()

'6c7999808f4f754ed0134427619d9c53c66254817461ead3d96ef1b2109930c1'

In [27]:
data = bytes([32])+a
r = "B0AC0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())

6c7999808f4f754ed0134427619d9c53c66254817461ead3d96ef1b2109930c1


In [28]:
pow(int.from_bytes(a,'big'),FP-2,FP).to_bytes(32,'big').hex()

'0519f16e777f4b5e2655de301420ef2198e8fd9fcc8b796dadf44b4011478f13'

In [29]:
data = bytes([32])+a
r = "B0AD0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())

0519f16e777f4b5e2655de301420ef2198e8fd9fcc8b796dadf44b4011478f13


In [30]:
pow(int.from_bytes(a,'big'),2,N).to_bytes(32,'big').hex()

'3e2402069b3dc34243b14c057b241fb93b5bab724bb431dd2f39556caf92a408'

In [31]:
data = bytes([32])+a
r = "B0AE0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


3e2402069b3dc34243b14c057b241fb93b5bab724bb431dd2f39556caf92a408


In [32]:
pow(int.from_bytes(a,'big'),3,N).to_bytes(32,'big').hex()

'3cd18b68b7ab968510c359260f30c714b420323366b20cc5387a0d32e971c3bf'

In [33]:
data = bytes([32])+a
r = "B0AF0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


3cd18b68b7ab968510c359260f30c714b420323366b20cc5387a0d32e971c3bf


In [34]:
pow(int.from_bytes(a,'big'),N-2,N).to_bytes(32,'big').hex()

'9e28897c00d2148d41e5e97ad66f07b86e099d1fe528e3699e4531aa1c8bea46'

In [35]:
data = bytes([32])+a
r = "B0B00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


9e28897c00d2148d41e5e97ad66f07b86e099d1fe528e3699e4531aa1c8bea46


In [36]:
pub = ec.PrivateKey(os.urandom(32)).get_public_key()
sec = pub.sec()
pub.compressed=False
sec_unc = pub.sec()
sec_unc.hex()

'047cf95ef9bec9fefe9c4b594c1af76842edb150d3aaa0dcf83ff47fc95e6204fd2824ab17874f80e5f2ed02713b76ee3c4ab4c5eaaadcb1e0494faea3e77bebf7'

In [37]:
data = bytes([65])+sec_unc
r = "B0B10000"+(bytes([len(data)])+data).hex().upper()
# print(r)
res = app.request(r)
print(res.hex())
res == sec

037cf95ef9bec9fefe9c4b594c1af76842edb150d3aaa0dcf83ff47fc95e6204fd


True

In [38]:
data = bytes([33])+sec
r = "B0B20000"+(bytes([len(data)])+data).hex().upper()
# print(r)
res = app.request(r)
print(res.hex())
res == sec_unc

047cf95ef9bec9fefe9c4b594c1af76842edb150d3aaa0dcf83ff47fc95e6204fd2824ab17874f80e5f2ed02713b76ee3c4ab4c5eaaadcb1e0494faea3e77bebf7


True

In [39]:
connection.disconnect()